# Writing a preprocessing script in python


## First thing we will always do is load our modules

In [1]:
import glob
import os
import pdb
import subprocess
import argparse
import datetime

### Often times we won't know all the modules we want to import right off the bat  but I like to make sure that as I am scripting I always put my modules at the top this allows others who may use my script to make sure they have all the necessary tools

## Now lets start by building a function that will hold all the commands we want to execute

```
def prepro():
    #do something cool
```

## We will make a function that will hold all of our global variables and our above function
## I personally like to call this main

```
def main():
    prepro()
```

## Finally we have our two functions and lastly we will call our main( ) which will execute both our global variables and our command function

In [9]:
def prepro(basedir):
    #Do something cool
    print('Hello data in the path '+basedir)
def main():
    #load in all the global variables prepro needs, right now this is just the path to the data
    basedir='/Users/gracer/Desktop/data' 
    prepro(basedir) #call prepro to do cool things 
    
main()#call main to execute all our globals then run our prepro function

Hello data in the path /Users/gracer/Desktop/data


##  What do we want the function to accomplish:

1. skull stripping
2. motion correction
  * creating motion regressors
  * creating framewise displacement regressor
  * a nice easy to read PDF/html?
3. re-orient?
4. trim extra TRs?

## Let's fill in our main( ) function first with the global variables we will need.

In [10]:
def main():
    basedir='/Users/gracer/Desktop/data'
    prepro()

## Anything you define in the main( ) function has to become an argument in the prepro( ) function. 

In [12]:
def prepro(basedir):
    print('Hello data in the path '+basedir)
def main():
    basedir='/Users/gracer/Desktop/data'
    prepro(basedir)

## Let's start with skull stripping using fsl's BET function. This is a linux based command so we are going to need to use a module to python to understand it.

## Normally at the command line we would run something like this:

```
bet input output [options]
```

## In python we can use the os module to run linux commands

```
os.system(bet input output -F)
```

In [8]:

#try running this 

print(os.system('echo $FSLDIR'))


#now look at the terminal you launched your jupyter notebook from


0


## next lets take a close look at the input and output we need. What will the input look like? What do we want the output to look like?


In [13]:
input='/Users/gracer/Desktop/data/<subject number>/func/<nifiti_file>'


## Each time we run this command the only things we really need to change are the subject number and the name of the nifti file

## Our subject numbers and nifti files use a predictable pattern! 
## We can use the glob module to find everything with a similar pattern. 
## Here we are going to use a wildcard character (*) to represent the portions of the subject number that differ.

In [10]:
input=glob.glob('/Users/gracer/Desktop/data/sub-*/func/sub-*.nii.gz')
print input[1:5]

['/Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-bart_bold_brain.nii.gz', '/Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-bart_bold_brain_mask.nii.gz', '/Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-bart_bold_brain_mcf.nii.gz', '/Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-rest_bold.nii.gz']


## glob has created a list with everything matching our pattern criteria. We can use any of python's list comprehension tools to further explore the list

In [11]:
len(input)

149

In [12]:
input[1]

'/Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-bart_bold_brain.nii.gz'

## We can also take any element from the list and make it a string. By making a string we can grab IDs or other parts of interest

In [13]:
x=input[1]
print('this is '+x)
y=x.split('/')
print (y)
sub=y[5]
print sub

this is /Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-bart_bold_brain.nii.gz
['', 'Users', 'gracer', 'Desktop', 'data', 'sub-10159', 'func', 'sub-10159_task-bart_bold_brain.nii.gz']
sub-10159


## Let's make this look a little nicer

In [14]:
sub=input[1].split('/')[5]
print(sub)

sub-10159


## Now we have the subject number but it looks like we have multiple tasks. How can we split an element from the list to get the task information and the subject information?

In [15]:
subtask=input[1].split('/')[7].split('.')[0]
#subtask=subtask.strip('.nii.gz')
print(subtask)

sub-10159_task-bart_bold_brain


In [16]:
output=subtask+'_brain'
print(output)

sub-10159_task-bart_bold_brain_brain


## Lets go back to our bet command in the os wrapper. We now have all the elements we need to execute it.

In [17]:
os.system('bet' x output '-F')

SyntaxError: invalid syntax (<ipython-input-17-c263dd53731b>, line 1)

## This is a problem, we have our input defined, but it looks like os.system is expecting a string argument. 
## We need to use another wildcare to pass our variables as strings! 

In [22]:
#os.system('bet' x output '-F')
os.system('bet %s %s -F'%(x, output))

0

## The %s is a placeholder for string variable
The **%** lets python know to look to the % sign outside the string for the variable of interest. 
We could also use this to pass **integers and floats using %i and %f** respectively.

## Now we have the ability to run bet through python on one subject.... but what about all the other scans.... ? 

![](https://i.imgur.com/itVtNcK.gif)

In [1]:
input=glob.glob('/Users/gracer/Desktop/data/sub-*/func/sub-*.nii.gz')

'''
this is a little long to type each time, 
and it is really easy to mess up the / formating 
'''

NameError: name 'glob' is not defined

## os.path.join( ) is super useful to quickly define paths. It will format strings into paths and allows us to use the %s 

In [24]:
#input=glob.glob('/Users/gracer/Desktop/data/sub-*/func/sub-*.nii.gz')
basedir='/Users/gracer/Desktop/data'
path=os.path.join(basedir,'sub-*','func','sub-*.niig.gz')
print(path)
input=glob.glob(os.path.join(basedir,'sub-*','func','sub-*.nii.gz'))
print(input[1:5])

/Users/gracer/Desktop/data/sub-*/func/sub-*.niig.gz
['/Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-bart_bold_bra.nii.gz', '/Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-bart_bold_bra_mask.nii.gz', '/Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-bart_bold_brain.nii.gz', '/Users/gracer/Desktop/data/sub-10159/func/sub-10159_task-bart_bold_brain_mask.nii.gz']


## Let's put this altogether into our function prepro( ) with a loop

In [ ]:
def prepro(basedir):
    for item in glob.glob(os.path.join(basedir,'sub-*','func','sub-*.nii.gz')):
        input=item
        output_path=item.strip('.nii.gz')
        output=output_path+('_brain')
        os.system("/usr/local/fsl/bin/bet %s %s -F"%(input, output))
        pdb.set_trace()
def main():
    basedir='/Users/gracer/Desktop/data'
    prepro(basedir)

In [ ]:
main()

> <ipython-input-25-807966444b94>(2)prepro()
-> for item in glob.glob(os.path.join(basedir,'sub-*','func','sub-*.nii.gz')):


## Ta Da!!! You have your first preprocessing script! 
### But wait... how do you make sure you don't end up running the same function on the same data over and over?
### Let's write in a check statement

## We can use os.path.exists( ) to check if we have already run BET, and tell our function to skip that subject
### This is useful if you have two people preprocessing data, or if something happens (aka your computer runs out of power) 

In [ ]:
def prepro(basedir):
    for item in glob.glob(os.path.join(basedir,'sub-*','func','sub-*.nii.gz')):
        input=item
        output_path=item.strip('.nii.gz')
        output=output_path+'_brain.nii.gz'
        print(output)
        pdb.set_trace()
        if os.path.exists(output):
            print(output_path+' is already stripped')
        else:
            os.system("/usr/local/fsl/bin/bet %s %s -F"%(input, output))
        #pdb.set_trace()
def main():
    basedir='/Users/gracer/Desktop/data'
    prepro(basedir)

In [ ]:
main()

## Now that we know how:
1. To make a set of functions
2. Set our global variables
3. Wrap our linux commands
4. Use glob to get all our subjects through wildcard matching
5. Loop through our list of subjects (from glob)
6. Use string comprehension to format file names
7. Use if/else loops to check for existing data


### Try writing a function to skull strip a T1w scan

In [2]:
def prepro(basedir, args, arglist, DATA):
#bet
    if args.STRIP==True:
        for sub in DATA:
            for nifti in glob.glob(os.path.join(sub,'func','sub-*_task-%s_bold.nii.gz')%(arglist['TASK'])):
                input=item
                output_path=item.strip('.nii.gz')
                output=output_path+'_brain.nii.gz'
                print(output)
            
                if os.path.exists(output):
                    print(output_path+' is already stripped')
                else:
                    os.system("/usr/local/fsl/bin/bet %s %s -F"%(input, output))
#bet rage
    if args.RAGE==True:
        print("starting bet rage")
        for sub in DATA:
            for input in glob.glob(os.path.join(sub,'anat','*T1w.nii.gz')):
                output=input.strip('.nii.gz')
                print(output)
                if os.path.exists(output+'_brain.nii.gz'):
                    print(output+' exists, skipping')
                else:
                    BET_OUTPUT=output+'_defaced'
                    x=("/usr/local/fsl/bin/bet %s %s -R"%(input, BET_OUTPUT))
                    print(x)
                    os.system(x)
   
def main(DATA):
    basedir='/Users/gracer/Desktop/data'
    parser=argparse.ArgumentParser(description='preprocessing')
    parser.add_argument('-task',dest='TASK',
                        default=False, help='which task are we running on?')
    parser.add_argument('-bet',dest='STRIP',action='store_true',
                        default=False, help='bet via fsl using defaults for functional images')
    parser.add_argument('-betrage',dest='RAGE',action='store_true',
                        default=False, help='bet via fsl using robust estimation for anatomical images')
    args = parser.parse_args()
    arglist={}
    for a in args._get_kwargs():
        arglist[a[0]]=a[1]
    print(arglist)
    prepro(basedir, args, arglist, DATA)


## Whoa what is all this extra stuff??? 
Let's take a look at all the new functionality that is possible 

## argparse
This will allow us to give user defined arguments
### Why would I make my life more complicated?

- This is a good option for if you have multiple people working on preprocessing 


- You want to make one giant preprocessing script and have control over which commands are run  
   
   Instead of a bunch of smaller functions that infividually have to be called 

In [2]:
def main(DATA):
    basedir='/Users/gracer/Desktop/data'
    parser=argparse.ArgumentParser(description='preprocessing')
    parser.add_argument('-task',dest='TASK',
                        default=False, help='which task are we running on?')
    parser.add_argument('-bet',dest='STRIP',action='store_true',
                        default=False, help='bet via fsl using defaults for functional images')
    parser.add_argument('-betrage',dest='RAGE',action='store_true',
                        default=False, help='bet via fsl using robust estimation for anatomical images')
    args = parser.parse_args()
    arglist={}
    for a in args._get_kwargs():
        arglist[a[0]]=a[1]
    print(arglist)
    prepro(basedir, args, arglist, DATA)


* Define the parser, and give it a nice description 

```
parser=argparse.ArgumentParser(description='preprocessing')

```

* Start thinking about arguments, these should be things that either trigger functionality or are global variables

```
parser.add_argument('-task',dest='TASK',
                        default=False, help='which task are we running on?')
parser.add_argument('-betrage',dest='RAGE',action='store_true',
                        default=False, help='bet via fsl using robust estimation for anatomical images')

```
1. The first part is the flag, it will be entered with our script at the commandline
2. Next is the destination, that is what the argparser is saving our input as
3. We set the default to false, meaning you have to enter something for it to be true
4. It is always nice to have a help especially if you are sharing your code!
5. Action will store your variable, this is nice for instances where the presences of the flag indicates true


* Put all our arguments in an variable called args

```
args = parser.parse_args()
```

* We could keep our arguments here and call them like this:

```
args.RAGE==True
```

* Personally I like putting them in a dictionary!

```
arglist={}
    for a in args._get_kwargs():
        arglist[a[0]]=a[1]
```

* Then you can call them in a dictionary form like this:

```
for nifti in glob.glob(os.path.join(sub,'func','sub-*_task-%s_bold.nii.gz')%(arglist['TASK'])):
```

* The key is 'TASK' and python will understand that to be what ever you entered into your argument list